#### OBJETIVO: Generacion de mapas mensuales a partir de los datos diarios

In [1]:
import numpy as np
import rasterio
from pathlib import Path
import glob

print("librerias ok ")

librerias ok 


In [5]:

estacion = "CH"
year = 2015
tipoModelo = "XGB"
modelo = "01-XGB-CV-M1-190625-CH"

ruta_diarios_base = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/salidas/SalidasDiarias/"
    f"{modelo}/{year}"
)

ruta_salida = (
    f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/salidas/SalidasMensuales_py/"
    f"{modelo}/base"
)

Path(ruta_salida).mkdir(parents=True, exist_ok=True)

months = [f"{i:02d}" for i in range(1, 13)]

for mes in months:

    print("Procesando mes:", mes)

    ruta_mes = f"{ruta_diarios_base}/{mes}"
    lista_rasters = sorted(glob.glob(ruta_mes + "/*.tif"))

    n_files = len(lista_rasters)
    print(f"  Archivos encontrados: {n_files}")

    if n_files == 0:
        print("  ⚠️ Mes vacío, se salta")
        continue

    stack = []

    # 🔹 leer rasters diarios
    for path in lista_rasters:
        with rasterio.open(path) as src:
            data = src.read(1)
            profile = src.profile

            nodata = src.nodata
            if nodata is not None:
                data = np.where(data == nodata, np.nan, data)

            stack.append(data)

    # 🔹 apilar y promediar
    stack = np.stack(stack, axis=0)  # (n_dias, rows, cols)
    promedio_mensual = np.nanmean(stack, axis=0)

    # 🔹 actualizar metadata
    profile.update(
        dtype=rasterio.float32,
        count=1,
        compress="lzw",
        nodata=np.nan
    )

    nombre_salida = (
        f"{ruta_salida}/{tipoModelo}_PM2.5_M_{mes}-{year}_{estacion}_V1.1.tif"
    )

    # 🔹 guardar raster mensual
    with rasterio.open(nombre_salida, "w", **profile) as dst:
        dst.write(promedio_mensual.astype(np.float32), 1)

    print("  ✔ Guardado:", nombre_salida)


Procesando mes: 01
  Archivos encontrados: 30
  ✔ Guardado: D:/Josefina/Proyectos/ProyectoChile/CH/modelos/salidas/SalidasMensuales_py/01-XGB-CV-M1-190625-CH/base/XGB_PM2.5_M_01-2015_CH_V1.1.tif
Procesando mes: 02
  Archivos encontrados: 25
  ✔ Guardado: D:/Josefina/Proyectos/ProyectoChile/CH/modelos/salidas/SalidasMensuales_py/01-XGB-CV-M1-190625-CH/base/XGB_PM2.5_M_02-2015_CH_V1.1.tif
Procesando mes: 03
  Archivos encontrados: 28
  ✔ Guardado: D:/Josefina/Proyectos/ProyectoChile/CH/modelos/salidas/SalidasMensuales_py/01-XGB-CV-M1-190625-CH/base/XGB_PM2.5_M_03-2015_CH_V1.1.tif
Procesando mes: 04
  Archivos encontrados: 27
  ✔ Guardado: D:/Josefina/Proyectos/ProyectoChile/CH/modelos/salidas/SalidasMensuales_py/01-XGB-CV-M1-190625-CH/base/XGB_PM2.5_M_04-2015_CH_V1.1.tif
Procesando mes: 05
  Archivos encontrados: 21
  ✔ Guardado: D:/Josefina/Proyectos/ProyectoChile/CH/modelos/salidas/SalidasMensuales_py/01-XGB-CV-M1-190625-CH/base/XGB_PM2.5_M_05-2015_CH_V1.1.tif
Procesando mes: 06
  Arch